In [27]:
import pyodbc
import json
from tqdm import tqdm_notebook

In [28]:
#Creating the connection (local)
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=localhost\SQLEXPRESS;'
                      'Database=pollution;'
                      'Trusted_Connection=yes;')

In [45]:
"""
This fonction inserts in bd elements of a json file
input : file recovery path
output: / 
"""

def insert_bd(path):
    #file reading
    with open(path, 'r') as f:
        datas = json.load(f)
    
    conn.commit()
    #Browse all articles
    for article in tqdm_notebook(datas.keys()):
        #Retrieving article informations
        author = datas[article]['authors'].replace("'", "")
        link = datas[article]['link']
        newspaper = datas[article]['newspaper'] 
        publication_date = datas[article]['publication_date']
        words = datas[article]['words'] 
        
        #Insertion in Newspapers
        insert_newspaper = conn.cursor()
        insert_newspaper.execute("exec InsertNewspaper @name_newspaper ='" + str(newspaper) +"';")   
        conn.commit()
        
        #Retrieving the id of the inserted newspaper
        select_newspaper = conn.cursor()
        select_newspaper.execute("SELECT id_newspaper FROM NEWSPAPERS WHERE NAME_NEWSPAPER = '" + str(newspaper) + "';")
        for id_news in select_newspaper:
            #Insertion in Articles
            insert_article = conn.cursor()
            insert_article.execute("exec InsertArticles @link_article = '" + str(link) + "', @publication_date = '" \
                                   + str(publication_date) + "', @authors = '" + str(author) + "', @id_newspaper = " \
                                   + str(id_news[0]) + ";")
        conn.commit()
        
        #Browse all words
        for word in words.keys():
            #We remove the quotation marks
            name_word = word.replace("'", "")
            
            #Retrieving word informations
            pos_tag = words[word]['pos_tag']
            tf = words[word]['tf']
            is_title = words[word]['is_title']
            is_description = words[word]['is_description']
            
            #Retrieving the id of the inserted article
            select_article = conn.cursor()
            select_article.execute("SELECT id_article FROM ARTICLES WHERE LINK_ARTICLE = '" + str(link) + "';")
            for j in select_article:
                id_article = j
            
            #Insertion in Words
            insert_word = conn.cursor()
            insert_word.execute("exec InsertWords @word = '" + str(name_word) + "', @pos_tag = '" + str(pos_tag) + "';")        
            conn.commit()
            
            #Retrieving the id of the inserted word
            select_word = conn.cursor()
            select_word.execute("SELECT id_word FROM WORDS WHERE WORD = '" + str(name_word) + "';")
            for id_word in select_word:
                #Insertion in Belong
                insert_belong = conn.cursor()
                insert_belong.execute("exec InsertBelong @id_article = %d, @id_word = %d, @tf = %d, @is_description = %d, @is_title = %d ;" \
                                      %(int(str(id_article[0])), int(str(id_word[0])), int(tf), int(is_description), int(is_title)))
            conn.commit()
    

7eme continent

In [46]:
insert_bd('../data/articles/clean_art_7econ.json')

Journal de l'environnement

In [47]:
insert_bd('../data/articles/clean_art_jdle.json')

Le Monde

In [ ]:
insert_bd('../data/articles/clean_art_lmde.json')

# <center>End</center>